<a href="https://colab.research.google.com/github/awannabecs/MY-FUNCTION-NAME/blob/main/PROJECT_1_TEXT_CLASSIFICATION_DEPRESSION_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import the libraries 
import numpy as  np
import pandas as pd 
import tensorflow as tf
import nltk 
import re
nltk.download("punkt")
tokenizer = nltk.word_tokenize
tokenizer("go out")
stemmer = nltk.PorterStemmer().stem
stemmer("played")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


'play'

In [2]:
#import the dataset
df = pd.read_csv("/content/drive/MyDrive/MAIN_DATA")
df.dropna(inplace=False)

,Unnamed: 0,Unnamed: 0.1,depressed,text
0,0,0.0,0.0,I know where you're coming from mate.
1,1,1.0,0.0,I felt like this last year.
2,2,2.0,1.0,"I remember around this time last year, i was h..."
3,3,3.0,1.0,"i was heavily addicted to meth, it was literal..."
4,4,4.0,0.0,"I worked like 70+hours in a warehouse job, 2 h..."
...,...,...,...,...
1790,178,1210.0,1.0,hate shattered me
1791,179,1212.0,1.0,but I know depression and drug addiction don'...
1792,180,1233.0,1.0,I feel a lot of deeply depressed souls have d...
1793,181,1234.0,1.0,anxiety and mental breakdowns...|0|0|0|


In [3]:
#create the word2number dictionary
tokens = []
for i in range(len(df)):
  words = []
  try:
    words.extend(tokenizer(df.iloc[i]["text"].lower()))
    for word in words:
      word = word
      tokens.append(stemmer(word))
  except Exception as e:
    print(df.iloc[i]["text"])

tokens = set(tokens)

nan
nan
nan
nan
nan


In [4]:
somelist = [" '//twitter.com/ltsharrypotter/status/984475138933579782',", '//t.co/yacazn2t0h',
            "dk_mok", '70+hour', 'much|1|0|4|', 'collection|1|0|3|',"20",]
pattern = """\/?\/?twitter.com/.+|
\/?\/?t.co\/?.+|  
_|
\d+|
\|d\|\d\|\d|
"""
for word in somelist:
  word = re.sub(pattern,"",word)
  print(word)

 '
//t.co/yacazn2t0h
dk_mok
70+hour
much|1|0|4|
collection|1|0|3|
20


In [5]:
token2index_dict = {"OOV":0}
index2token_dict = {0:"OOV"}
for index,token in enumerate(tokens):
  token2index_dict[token] = index+1
  index2token_dict[index+1] = token

In [6]:
def token2index_function(dict,sent):
  result_sent = []
  for token in sent.split(" "):
    result_sent.append(dict.get(token,0))
  return result_sent


def index2token_function(dict,index):
  return dict.get(index,"OOV")

In [7]:
#make the sentences same-lengthed 
#find the max length
max_length = 0
for sent in range(len(df)):
  try:
    words = nltk.word_tokenize(df.iloc[sent]["text"])
  except Exception as e:
    print(df.iloc[sent]["text"])
  if len(words) > max_length:
    max_length = len(words)

nan
nan
nan
nan
nan


In [8]:
vector = [1,2,3,4,5,6,7,8]
def same_lengther(vector,max_length):
  zeros = np.zeros(max_length - len(vector))
  vector.extend(zeros)
  return vector

In [9]:
#define X and y
X = []
y = []
X_vector = []


for i in range(len(df)):
  feature = df["text"].iloc[i]
  label = int(df["depressed"].iloc[i])

  try:
    feature_vector = token2index_function(token2index_dict,feature)
    X_vector.append(same_lengther(feature_vector,max_length))
    y.append(label)
    X.append(feature)
  except Exception as e:
    print(i)
    


534
727
979
1019
1242


In [75]:
y = np.array(y).reshape(-1,1)
X_vector = np.array(X_vector)

new_X_vector = []
new_y = []
for i in range(len(y)):
  new_X_vector.append(X_vector[i].reshape(-1,1))
  new_y.append(y[i].reshape(-1,1))

new_X_vector = np.array(new_X_vector)
new_y = np.array(new_y)

In [76]:
# new_X_vector = np.squeeze(new_X_vector)
# new_X_vector[0].shape
new_y.shape,new_y[0].shape,new_X_vector.shape,new_X_vector[0].shape

((2565, 1, 1), (1, 1), (2565, 121, 1), (121, 1))

In [77]:
tf.random.set_seed(666)
model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.Dense(1,activation="relu"))
model1.add(tf.keras.layers.Dense(1,activation="relu"))
model1.add(tf.keras.layers.Dense(1,activation="relu"))
model1.add(tf.keras.layers.Dense(2,activation="sigmoid"))

model1.compile(loss=tf.keras.losses.binary_focal_crossentropy,
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

model1.fit(X_vector,y,epochs=5)

Epoch 1/5


ValueError: ignored